In [1]:
import numpy as np
import time
import string

**1**. (25 points)

The following iterative sequence is defined for the set of positive integers:

- n → n/2 (n is even)
- n → 3n + 1 (n is odd)

Using the rule above and starting with 13, we generate the following sequence:

13 → 40 → 20 → 10 → 5 → 16 → 8 → 4 → 2 → 1

It can be seen that this sequence (starting at 13 and finishing at 1) contains 10 terms. Although it has not been proved yet (Collatz Problem), it is thought that all starting numbers finish at 1.

Which starting number, under one million, produces the longest chain?

NOTE: Once the chain starts the terms are allowed to go above one million.

**Straightforward Solution**: Loop through 1 - 1,000,000, and keep track of the maximum length of chain and corresponding starting number.

In [2]:
%%time

def length(starting_num):
    l = 1
    while starting_num != 1:
        starting_num = starting_num // 2 if starting_num % 2 == 0 else starting_num * 3 + 1
        l += 1
    return l

maximum_len, maximum_start = None, None
for num in range(1, 1000001):
    cur_len = length(num)
    if maximum_len is None or cur_len > maximum_len: # cannot change order: short circuit evaluation of `or`
        maximum_len, maximum_start = cur_len, num
        
print(maximum_start)

837799
CPU times: user 17.8 s, sys: 8 ms, total: 17.8 s
Wall time: 17.8 s


**Recursion:** The recursion can be formulated as $$\texttt{chain-len}(i) = 1 + \left\{\begin{aligned}\texttt{chain-len}(i // 2)\quad i\text{ is even};\\\texttt{chain-len}(i * 3 + 1)\quad i\text{ is odd}.
\end{aligned}\right.$$

In [3]:
%%time

# num_function_calls = 0

def length(starting_num):
    # global num_function_calls
    # num_function_calls += 1
    if starting_num == 1:
        return 1
    elif starting_num % 2 == 0:
        return 1 + length(starting_num // 2)
    else:
        return 1 + length(starting_num * 3 + 1)

maximum_len, maximum_start = None, None
for num in range(1, 1000001):
    cur_len = length(num)
    if maximum_len is None or cur_len > maximum_len: # cannot change order: short circuit evaluation of `or`
        maximum_len, maximum_start = cur_len, num

print(maximum_start)
# print(f'Total function calls: {num_function_calls}')

837799
Total function calls: 132434424
CPU times: user 43.6 s, sys: 20 ms, total: 43.6 s
Wall time: 43.6 s


Recursion is slower as a result of many function calls. However, recursion often helps in simplifying complex problems.

**Faster Solution:** We can actually make use of the information we obtained in previous computation.

Take the following example: Suppose we have calculated `chain-len(10) = 7`: 10 → 5 → 16 → 8 → 4 → 2 → 1. We can speed up the calculation in two ways:

1) We should not need to calculate `chain-len(16)`;

2) We should not need to calculate `chain-len(13)` to the last step (13 → 40 → 20 → 10 → 5 → 16 → 8 → 4 → 2 → 1).

In [4]:
%%time

# num_function_calls = 0
chain_len_dict = {1: 1}

def length(starting_num):
    # global num_function_calls
    num_function_calls += 1
    
    # already calculated: just return the value
    if starting_num in chain_len_dict.keys():
        return chain_len_dict[starting_num]
    
    new_num = starting_num // 2 if starting_num % 2 == 0 else starting_num * 3 + 1
    # if the length of chain from new_num is l, then the length from starting_num is l+1.
    result = 1 + length(new_num) 
    
    # cache the result
    chain_len_dict[starting_num] = result
    return result

maximum_len, maximum_start = None, None
for num in range(1, 1000001):
    cur_len = length(num)
    if maximum_len is None or cur_len > maximum_len: # cannot change order: short circuit evaluation of `or`
        maximum_len, maximum_start = cur_len, num

print(maximum_start)
# print(f'Total function calls: {num_function_calls}')

837799
Total function calls: 3168610
CPU times: user 2.24 s, sys: 116 ms, total: 2.35 s
Wall time: 2.35 s


**Optional: only for your interest**

- List: `a = [..., ..., ..., ..., ..., ...]`. Use `a[i]` to denote the value of `chain-len(i)`.

- Dict: `a = {...:..., ...:..., ...:..., ...:...}`. When you visit `i`, put `i:chain-len(i)` into the dictionary.

In Python, searching whether an element is in a dictionary (based on hashmap) and fetching the key takes $O(1)$ to $O(n)$ time (depending on how many elements are in the set), while searching whether an element is in a list takes $O(n)$ time. Therefore, we prefer using a dictionary to store the visited elements.

However, taking an element out from a list using index only takes $O(1)$ time. Therefore, for the list implementation, we do not simply put visited elements in a list; instead, we create a big list where each element corresponds to an index. This is called as "exchanging space for time".

**Fact 1**: We will sooner or later have calculated all `chain-len(i)` for `1 <= i <= 1000000`;

**Fact 2**: We may encounter some `chain-len(i)` values where `i` is larger than `1000000`, and we do not yet know how much the number can be.

Due to fact 2, we cannot pre-assign a list and use `list[i]` to store the value of `chain-len(i)`. (We even don't know how large the list should be. Even if we know an upper bound, this may also be a waste of memory space.)

Due to fact 1, if we use a dictionary to store `i` and corresponding `chain-len(i)`, the dictionary will at least have 1000000 elements, which might make searching slower.

A compromise is to use a list for `1 <= i <= 1000000` and use a dictionary for other values encountered.

In [5]:
%%time

chain_len_list = [None] * 1000001
chain_len_list[1] = 1 # Why we need this?
chain_len_dict = {}

def length(starting_num):
    # already calculated: just return the value
    if starting_num <= 1000000 and chain_len_list[starting_num]: 
        return chain_len_list[starting_num]
    if starting_num > 1000000 and starting_num in chain_len_dict.keys():
        return chain_len_dict[starting_num]
    
    new_num = starting_num // 2 if starting_num % 2 == 0 else starting_num * 3 + 1
    # if the length of chain from new_num is l, then the length from starting_num is l+1.
    result = 1 + length(new_num) 
    
    # cache the result
    if starting_num <= 1000000:
        chain_len_list[starting_num] = result
    else:
        chain_len_dict[starting_num] = result
    return result

maximum_len, maximum_start = None, None
for num in range(1, 1000001):
    cur_len = length(num)
    if maximum_len is None or cur_len > maximum_len: # cannot change order: short circuit evaluation of `or`
        maximum_len, maximum_start = cur_len, num
print(maximum_start)

837799
CPU times: user 1.73 s, sys: 84 ms, total: 1.82 s
Wall time: 1.81 s


**2** (25 points)


- Perform the median polish to calculate just the *residuals* for this [example](https://mgimond.github.io/ES218/Week11a.html) in Python. 
- Use the matrix `xs` provided
- Display the final result after 3 iterations to 1 decimal place and check if it agrees with 

![img](https://mgimond.github.io/ES218/img/twoway_09.jpg)

In [6]:
xs = np.array([
    (25.3,32.1,38.8,25.4), 
    (25.3,29,31,21.1),
    (18.2,18.8,19.3,20.3),
    (18.3,24.3,15.7,24),
    (16.3,19,16.8,17.5)
]).T

In [7]:
def loop(array, nloop):
    nrow, ncol = array.shape
    
    # Step 1: Compute overall median and residual table
    overall_median = np.median(array)
    residual_table = array - overall_median # broadcast
    row_effect = np.zeros(nrow)
    col_effect = np.zeros(ncol)
    
    for i in range(nloop):
        # Step 2: Compute the row medians
        row_medians = np.median(residual_table, axis = 1)
        med_col_effect = np.median(col_effect)

        # Step 3: Create a new residual table from the row medians
        row_effect += row_medians
        overall_median += med_col_effect
        col_effect -= med_col_effect
        residual_table -= row_medians[:, None]
        
        # Step 4: Compute the column medians
        col_medians = np.median(residual_table, axis = 0)
        med_row_effect = np.median(row_effect)
        
        # Step 5: Create a new residual table from the column medians
        col_effect += col_medians
        overall_median += med_row_effect
        row_effect -= med_row_effect
        residual_table -= col_medians
    
    return np.round(residual_table, 1)

loop(xs, 3)

array([[-1.4,  0.2,  0. , -1. ,  0.7],
       [ 1.4, -0.2, -3.4,  1. , -0.7],
       [11. ,  4.7, -0. , -4.7,  0. ],
       [-3.1, -5.9,  0.3,  2.9,  0. ]])

In [8]:
def loop(array, nloop):
    overall_median = np.median(array)
    residual_table = array - overall_median
    for i in range(3):
        row_medians = np.median(residual_table, axis = 1)
        residual_table -= row_medians[:, None]
        col_medians = np.median(residual_table, axis = 0)
        residual_table -= col_medians
    return np.round(residual_table, 1)

loop(xs, 3)

array([[-1.4,  0.2,  0. , -1. ,  0.7],
       [ 1.4, -0.2, -3.4,  1. , -0.7],
       [11. ,  4.7, -0. , -4.7,  0. ],
       [-3.1, -5.9,  0.3,  2.9,  0. ]])

**3**. (50 points)

A Caesar cipher is a very simple method of encoding and decoding data. The cipher simply replaces characters with the character offset by $k$ places. For example, if the offset is 3, we replace `a` with `d`, `b` with `e` etc. The cipher wraps around so we replace `y` with `b`, `z` with `c` and so on. Punctuation, spaces and numbers are left unchanged.

- Write a function `encode` that takes as arguments a string and an integer offset and returns the encoded cipher.
- Write a function `decode` that takes as arguments a cipher and an integer offset and returns the decoded string. 
- Write a function `auto_decode` that takes as argument a cipher and uses a statistical method to guess the optimal offset to decode the cipher, assuming the original string is in English which has the following letter frequency:

```python
freq = {
 'a': 0.08167,
 'b': 0.01492,
 'c': 0.02782,
 'd': 0.04253,
 'e': 0.12702,
 'f': 0.02228,
 'g': 0.02015,
 'h': 0.06094,
 'i': 0.06966,
 'j': 0.00153,
 'k': 0.00772,
 'l': 0.04025,
 'm': 0.02406,
 'n': 0.06749,
 'o': 0.07507,
 'p': 0.01929,
 'q': 0.00095,
 'r': 0.05987,
 's': 0.06327,
 't': 0.09056,
 'u': 0.02758,
 'v': 0.00978,
 'w': 0.0236,
 'x': 0.0015,
 'y': 0.01974,
 'z': 0.00074
}
```

- Encode the following nursery rhyme using a random offset from 10 to 20, then recover the original using `auto_decode`:

```text
Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;
One for the master,
And one for the dame,
And one for the little boy
Who lives down the lane.
```

In [9]:
def encode(s, offset):
    offset = offset % 26 # let offset be within 0 and 25 (mod 26)
    shift = lambda letters: letters[offset:] + letters[:offset] 
    # offset is defined outside shift, so it can be used in the lambda expression.
    # offset is similar to global variable for shift
    all_letters = string.ascii_lowercase + string.ascii_uppercase
    shifted_letters = shift(string.ascii_lowercase) + shift(string.ascii_uppercase)
    return s.translate(str.maketrans(all_letters, shifted_letters))

$-30 = -2 \times 26 + 22$ (python, R) - round down the quotient

$-30 = -1 \times 26 + (-4)$ (C, C++) - round the quotient towards zero

In both cases, you can write `offset = (offset % 26 + 26) % 26` to get non-negative remainders.

In [10]:
def decode(s, offset):
    return encode(s, -offset)

How can we determine what offset the rhyme is encoded? We can do this based on the frequencies of each letter.

An easy way is to identify the most common letter as `'e'`, since it has the highest frequency of appearance.

If you suspect that only using one letter might be subject to randomness,
- Theoretical frequency: $f = (f_1, f_2, \cdots, f_{26})$.

- Empirical frequency under offset $i$: $f^{(i)} = (f_1^{(i)}, f_2^{(i)},\cdots, f_{26}^{(i)})$.

Find $i$ such that $f^{(i)}$ is the closest to $f$.

$f$ and $f^{(i)}$ are two points in $\mathbb{R}^{26}$, or the 25-simplex $\Delta^{25}$.

Metrics:

- inner product: $d_i = \sum f_jf_j^{(i)}$;

- $L_2$-norm: $d_i^2 = \sum (f_j - f_j^{(i)})^2$;

Kullback-Leibler divergence: $D_{KL}(f || f^{(i)})$.

In [11]:
freq = {
 'a': 0.08167,
 'b': 0.01492,
 'c': 0.02782,
 'd': 0.04253,
 'e': 0.12702,
 'f': 0.02228,
 'g': 0.02015,
 'h': 0.06094,
 'i': 0.06966,
 'j': 0.00153,
 'k': 0.00772,
 'l': 0.04025,
 'm': 0.02406,
 'n': 0.06749,
 'o': 0.07507,
 'p': 0.01929,
 'q': 0.00095,
 'r': 0.05987,
 's': 0.06327,
 't': 0.09056,
 'u': 0.02758,
 'v': 0.00978,
 'w': 0.0236,
 'x': 0.0015,
 'y': 0.01974,
 'z': 0.00074
}

In [12]:
rhyme = '''
    Baa, baa, black sheep,
    Have you any wool?
    Yes, sir, yes, sir,
    Three bags full;
    One for the master,
    And one for the dame,
    And one for the little boy
    Who lives down the lane.
'''

In [13]:
encoded_rhyme = encode(rhyme, np.random.randint(10, 21))

In [14]:
print(encoded_rhyme)


    Mll, mll, mwlnv dsppa,
    Slgp jzf lyj hzzw?
    Jpd, dtc, jpd, dtc,
    Escpp mlrd qfww;
    Zyp qzc esp xldepc,
    Lyo zyp qzc esp olxp,
    Lyo zyp qzc esp wteewp mzj
    Hsz wtgpd ozhy esp wlyp.



In [15]:
from collections import Counter
import re
l = Counter(re.sub(r'[^a-zA-Z]', '', encoded_rhyme.lower())).most_common(1)[0][0] # find the most common letter
offset = ord(l) - ord('e')
decoded_rhyme = decode(encoded_rhyme, offset)

In [16]:
print(decoded_rhyme)


    Baa, baa, black sheep,
    Have you any wool?
    Yes, sir, yes, sir,
    Three bags full;
    One for the master,
    And one for the dame,
    And one for the little boy
    Who lives down the lane.

